<a href="https://colab.research.google.com/github/Elwing-Chou/tibame1224/blob/main/0114_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HTTP代碼

```python
2開頭: 成功

3開頭: 轉址
對面的網址已經搬家了, 雖然妳還是輸入舊網址, 但她會自動幫你替換成新網址

4開頭: 錯誤

404: Not Found
!!! 最常見: 網址打錯
403: Forbidden
!!! 原因1: ip ban
!!! 通常是數次的惡意行為(可能是短時間送太多...)
!!! 解法: 等待/換ip
!!! 原因2: 妳學的不像瀏覽器
!!! 解法: 學的像一點(headers完善)

```

Header

```python
妳在送出網址的時候, 其實不只送出網址
妳送出的是 網址+額外信息(request headers)

妳在得到回應的時候, 其實不是只有得到回應
回應+額外信息(response headers)
```

處理檔案

```python
1. 純文字(TXT)
open(...., "r" or "w", encoding="utf-8")

2. 非純文字(圖片/pdf/doc....)
open(...., "rb" or "wb")

```

```python
!!!! 下次檢討

把整頁的的文章的圖片都下載, 並且放在不同資料夾裡
!!! 網址是縮寫的
!!! 有些文章是被刪文的
```

In [ ]:
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

response = req.urlopen(r)
html = bs.BeautifulSoup(response)

# 如果沒有class的話
# 1. 找她老爸
# 2. 自己寫篩選方式(!!!!!)
allow_subname = ["jpg", "jpeg", "png", "gif"]
links = html.find_all("a")
for l in links:
    href = l["href"]
    subname = href.split(".")[-1]
    if subname.lower() in allow_subname:
        r = req.Request(href)
        r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")
        img = req.urlopen(r)
        # open一個新檔案來儲存
        fname = href.split("/")[-1]
        f = open(fname, "wb")
        f.write(img.read())
        f.close()
        # print(img.read())
        # print("-" * 30)

In [ ]:
# open
import os
# 如果這個資料夾不存在
# 為何是os.path->下次說
if not os.path.exists("test/test2"):
    # 把它創造出來
    os.makedirs("test/test2")

f = open("test/test2/a.txt", "w", encoding="utf-8")
f.write("abcede")
f.close()

In [ ]:
# 練習: 拿其他的
import os
import json
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1736733511.A.738.html"
# 順便準備一下等一下要儲存的檔案名稱
url_split = url.split("/")
fname = url_split[-1] + ".json"
dirname = url_split[-2]
# 記得要先創資料夾, 不然紅字
if not os.path.exists(dirname):
    os.makedirs(dirname)
fpath = dirname + "/" + fname

r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

response = req.urlopen(r)
html = bs.BeautifulSoup(response)

meta_list = html.find_all("span", {"class":"article-meta-value"})

ptt_id = meta_list[0].text
if "(" in ptt_id:
    ptt_id_split = ptt_id.split("(")
    ptt_id = ptt_id_split[0].strip()
    nick_name = ptt_id_split[-1].replace(")", "")
else:
    nick_name = ""
board = meta_list[1].text
title = meta_list[2].text
if title[0] == "[":
    title_split = title.split("]")
    category = title_split[0].replace("[", "")
    title = title_split[-1].strip()
else:
    category = ""
date = meta_list[3].text

# 存檔的時候肯定是來個JSON
data = {
    "id":ptt_id,
    "nickname":nick_name,
    "board":board,
    "category":category,
    "title":title,
    "data":date,
    "push":[]
}

print("作者:", ptt_id)
print("當時暱稱:", nick_name)
print("看板:", board)
print("分類:", category)
print("標題:", title)
print("時間:", date)


push_list = html.find_all("div", {"class":"push"})
for push in push_list:
    push_meta = push.find_all("span")
    push_type = push_meta[0].text.strip()
    push_ptt_id = push_meta[1].text.strip()
    push_content = push_meta[2].text.replace(": ", "").strip()
    push_ip_date = push_meta[3].text.strip()
    print(push_type, push_ptt_id, push_content, push_ip_date)
    push_data = {
        "type":push_type,
        "id":push_ptt_id,
        "content":push_content,
        "ipdate":push_ip_date
    }
    data["push"].append(push_data)

# 把json寫入到檔案裡
f = open(fpath, "w", encoding="utf-8")
# 之前是f.write, 現在使用json.dump幫我做更好的write
json.dump(data, f, ensure_ascii=False, indent=4)
f.close()

作者: JANUARZ
當時暱稱: 社會職人
看板: Beauty
分類: 正妹
標題: 有些角度像楊謹華
時間: Mon Jan 13 09:58:29 2025
推 ninaman 正 101.12.146.84 01/13 10:05
推 deltarobot . 49.217.122.9 01/13 10:07
推 elfindor 優 223.137.175.186 01/13 10:16
推 wglhe 優版派克 42.77.77.102 01/13 10:17
推 Uncontinue 正正 122.118.35.37 01/13 10:38
推 Williamtsou 門 27.53.230.154 01/13 10:52
推 aass5566 就是本人 42.77.55.73 01/13 11:10
→ bingreen 正 111.184.234.172 01/13 11:17
推 openbook13 優質皮朋 49.215.58.105 01/13 11:29
推 okbon 不錯喔 118.231.152.241 01/13 13:03
噓 TopGun2 明明就是 張鳳書 114.33.106.198 01/13 13:12
→ wl00669773 之前現場聊天過，一直覺得本人比照片 223.141.125.149 01/13 13:26
→ wl00669773 漂亮 223.141.125.149 01/13 13:27
噓 hmt17 許光漢？ 49.216.52.98 01/13 14:26
推 WasJohnWall 派克？ 61.228.67.71 01/13 15:32
推 durian0308 讚 42.78.236.55 01/13 15:51
推 saw6904 楊謹華天花版 42.79.150.39 01/13 18:24
推 ruffryders 正翻 42.78.17.7 01/13 19:57
推 a3300689 已追蹤 推藏頭 1.171.153.189 01/14 00:32
→ a9564208 比較像 常威 101.9.96.122 01/14 07:33
推 clkdtm32 比楊謹華正太多了吧 198.232.126.7 01/14 11:00


```python

聽故事!!!! 稍微知道就可以了

我們之前說import你就是要import到某個.py

照抄.功能() 照抄...

我們從來沒遇過妳某隻檔案裏面

os(檔案).xxxx.功能

os.py 兩行

import posixpath(檔案) as path
import ntpath(檔案) as path

這個path就是這個
!!! 因為外部使用者並不會想自己分辨
!!! 封裝(讓外部使用者感覺都是同一個exists)
!!! 但裡面幫妳導到不同作業系統的指令
!!! 妳外部永遠只要同一行 os.path.exists


在ntpath和posixpath裡面都有一行
from genericpath import *

事實上妳是吃到genericpath.py裡面的exists

exists是存在在genericpath.py裡面的

!!! 因為exists這指令在windows/mac/linux都是一樣的
!!! 那我們絕對不會重複一直寫, 所以我們就拉出一隻檔案把它定義裡面



```

In [1]:
# 01/20 加入創資料夾
import os
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html"
# 創造資料夾
dirname = url.split("/")[-1]
if not os.path.exists(dirname):
    os.makedirs(dirname)

r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

response = req.urlopen(r)
html = bs.BeautifulSoup(response)

allow_subname = ["jpg", "jpeg", "png", "gif"]
links = html.find_all("a")
for l in links:
    href = l["href"]
    subname = href.split(".")[-1]
    if subname.lower() in allow_subname:
        r = req.Request(href)
        r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")
        img = req.urlopen(r)
        fname = href.split("/")[-1]
        # 完整路徑(用字串加法是有點累的 "/", os.path.join不用)
        fpath = os.path.join(dirname, fname)
        f = open(fpath, "wb")
        f.write(img.read())
        f.close()


In [8]:
import os
import urllib.request as req
import bs4 as bs

def dl_post_image(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1736755829.A.02A.html"
    # 創造資料夾
    dirname = url.split("/")[-1]
    dirname = os.path.join("ptt", dirname)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

    response = req.urlopen(r)
    html = bs.BeautifulSoup(response)

    allow_subname = ["jpg", "jpeg", "png", "gif"]
    links = html.find_all("a")
    for l in links:
        href = l["href"]
        subname = href.split(".")[-1]
        if subname.lower() in allow_subname:
            r = req.Request(href)
            r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")
            img = req.urlopen(r)
            fname = href.split("/")[-1]
            # 完整路徑(用字串加法是有點累的 "/", os.path.join不用)
            fpath = os.path.join(dirname, fname)
            f = open(fpath, "wb")
            f.write(img.read())
            f.close()

In [9]:
# 當成全新的在寫
import urllib.request as req

url = "https://www.ptt.cc/bbs/Beauty/index3950.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0")

response = req.urlopen(r)
html = bs.BeautifulSoup(response)
titles = html.find_all("div", {"class":"title"})

for t in titles:
    post_link = t.find("a")
    # t裡有a: 正常
    if not post_link == None:
        post_url = "https://www.ptt.cc" + post_link["href"]
        print(post_url)
        dl_post_image(post_url)
    # t裡沒a: 刪文的
    else:
        print("這篇被刪了")

https://www.ptt.cc/bbs/Beauty/M.1737162050.A.EA1.html
https://www.ptt.cc/bbs/Beauty/M.1737164418.A.B9B.html
https://www.ptt.cc/bbs/Beauty/M.1737164759.A.625.html
這篇被刪了
https://www.ptt.cc/bbs/Beauty/M.1737172640.A.4C5.html
https://www.ptt.cc/bbs/Beauty/M.1737177354.A.3CF.html
https://www.ptt.cc/bbs/Beauty/M.1737179262.A.1E8.html
https://www.ptt.cc/bbs/Beauty/M.1737182986.A.920.html
https://www.ptt.cc/bbs/Beauty/M.1737189398.A.CAA.html
https://www.ptt.cc/bbs/Beauty/M.1737192599.A.90F.html
https://www.ptt.cc/bbs/Beauty/M.1737193583.A.A22.html
https://www.ptt.cc/bbs/Beauty/M.1737209212.A.D66.html
https://www.ptt.cc/bbs/Beauty/M.1737248469.A.BC8.html
https://www.ptt.cc/bbs/Beauty/M.1737248570.A.E9D.html
https://www.ptt.cc/bbs/Beauty/M.1737248677.A.D05.html
https://www.ptt.cc/bbs/Beauty/M.1737248796.A.252.html
https://www.ptt.cc/bbs/Beauty/M.1737251853.A.27E.html
https://www.ptt.cc/bbs/Beauty/M.1737256223.A.C91.html
https://www.ptt.cc/bbs/Beauty/M.1737258056.A.744.html
https://www.ptt.cc/bbs